In [1]:
import numpy as np
import random
import networkx as nx
import math
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
def find_roster(a :np.array, tol=0.001) -> np.array: #a: a non-negative fractions add up to 1
    n=len(a)
    m=1
    while ( ( ((np.mod(m*a, 1) > tol) & (np.mod(m*a, 1) < (1-tol)) )  *1).sum() > 0):
        m+=1
    
    P=np.zeros((m,n))
    for j in range(n):
        P[0:m,j]=a[j]
    
    G = nx.DiGraph()
    for i in range(m):
        for j in range(n):
            if i>1:
                G.add_edge((i,j,1), (i-1,j,1), weight=a[j]*i)
                G.add_edge((i,j,1), (i,j,0), weight=a[j])
            elif i==1:
                G.add_edge((i,j,1), (i-1,j,0), weight=a[j])
                G.add_edge((i,j,1), (i,j,0), weight=a[j])

            G.add_edge((i,j,0),(i,n,1),weight=a[j]) 
    G_frac = nx.DiGraph(((source, target, attr) 
                         for source, target, attr in G.edges(data=True)
                         if (np.mod(attr['weight'],1) > tol) & ((np.mod(attr['weight'],1) < (1- tol)))))
    
    while list(G_frac.nodes()): 
        cycle= nx.find_cycle(G_frac, orientation="ignore", 
                             source= list(G_frac.nodes())[np.random.choice(range(len(G_frac.nodes)))])
        
        d_minus = 1
        d_plus = 1
        for (source, target, direction) in cycle:
            weight= G_frac.get_edge_data(source,target)['weight']
            if direction == 'forward':
                d_plus= min((1-np.mod(weight, 1)),d_plus)
                d_minus= min(np.mod(weight, 1),d_minus)
            if direction == 'reverse':
                d_plus= min(np.mod(weight, 1),d_plus)
                d_minus= min((1-np.mod(weight, 1)),d_minus)
                
        p_minus=d_plus/(d_plus+d_minus)
        p_plus=d_minus/(d_plus+d_minus)

        select_minus= np.random.choice([1,0], 1, p=[p_minus,p_plus] )
        if select_minus:
            for (source, target, direction) in cycle:
                weight= G_frac.get_edge_data(source,target)['weight']
                if direction == 'forward':
                    G.add_edge(source, target, weight=weight-d_minus)
                if direction == 'reverse':
                    G.add_edge(source, target, weight=weight+d_minus)
        else:
            for (source, target, direction) in cycle:
                weight= G_frac.get_edge_data(source,target)['weight']
                if direction == 'forward':
                    G.add_edge(source, target, weight=weight+d_plus)
                if direction == 'reverse':
                    G.add_edge(source, target, weight=weight-d_plus)
        
        G_frac = nx.DiGraph(((source, target, attr) 
                     for source, target, attr in G.edges(data=True)
                     if (np.mod(attr['weight'],1) > tol) & ((np.mod(attr['weight'],1) < (1- tol)))))
        
    for i in range(m):
        for j in range(n):
            P[i,j]=G.get_edge_data((i, j, 0),(i, n, 1))['weight']
    P= np.round(P)
    return P

# Example

In [4]:
a=np.array([0.27,0.15,0.075,0.505])
print(a)

[0.27  0.15  0.075 0.505]


In [5]:
tol=0.0001
n=len(a)
m=1
while ( ( ((np.mod(m*a, 1) > tol) & (np.mod(m*a, 1) < (1-tol)) )  *1).sum() > 0):
    m+=1

P=np.zeros((m,n))
for j in range(n):
        P[0:m,j]=a[j]

print(P)
print('\n')
print(find_roster(a))

[[0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 [0.27  0.15  0.075 0.505]
 

In [11]:
T=5
simulation=np.zeros((T,m,n))

for i in range(T):
    simulation[i]= find_roster(a)

mean_simulation=np.average(simulation,axis=0)
print(mean_simulation)

[[0.2 0.2 0.2 0.4]
 [0.4 0.  0.  0.6]
 [0.4 0.  0.2 0.4]
 [0.  0.4 0.  0.6]
 [0.4 0.  0.2 0.4]
 [0.2 0.2 0.  0.6]
 [0.4 0.2 0.  0.4]
 [0.  0.2 0.2 0.6]
 [0.2 0.2 0.  0.6]
 [0.4 0.2 0.  0.4]
 [0.4 0.2 0.  0.4]
 [0.4 0.  0.  0.6]
 [0.  0.  0.2 0.8]
 [0.6 0.2 0.  0.2]
 [0.  0.4 0.  0.6]
 [0.6 0.  0.  0.4]
 [0.  0.  0.  1. ]
 [0.2 0.4 0.2 0.2]
 [0.2 0.2 0.  0.6]
 [0.4 0.  0.2 0.4]
 [0.2 0.2 0.  0.6]
 [0.2 0.2 0.2 0.4]
 [0.4 0.2 0.  0.4]
 [0.  0.4 0.  0.6]
 [0.6 0.  0.  0.4]
 [0.2 0.  0.4 0.4]
 [0.6 0.2 0.  0.2]
 [0.  0.2 0.  0.8]
 [0.  0.  0.2 0.8]
 [0.6 0.2 0.  0.2]
 [0.4 0.4 0.  0.2]
 [0.2 0.  0.  0.8]
 [0.2 0.  0.2 0.6]
 [0.4 0.2 0.  0.4]
 [0.2 0.2 0.  0.6]
 [0.4 0.  0.2 0.4]
 [0.  0.2 0.2 0.6]
 [0.  0.  0.2 0.8]
 [0.6 0.  0.  0.4]
 [0.4 0.4 0.  0.2]
 [0.  0.4 0.  0.6]
 [0.4 0.  0.  0.6]
 [0.4 0.  0.  0.6]
 [0.2 0.  0.  0.8]
 [0.  0.6 0.  0.4]
 [0.4 0.  0.2 0.4]
 [0.6 0.  0.  0.4]
 [0.  0.4 0.  0.6]
 [0.6 0.  0.  0.4]
 [0.  0.2 0.2 0.6]
 [0.2 0.  0.2 0.6]
 [0.2 0.  0.  0.8]
 [0.4 0.4 0.

In [7]:
print(P-mean_simulation)

[[-0.13   0.15  -0.125  0.105]
 [-0.13   0.15   0.075 -0.095]
 [ 0.27  -0.05  -0.125 -0.095]
 [ 0.07  -0.25   0.075  0.105]
 [ 0.27  -0.25  -0.125  0.105]
 [-0.13   0.15   0.075 -0.095]
 [-0.33   0.15   0.075  0.105]
 [ 0.07   0.15  -0.125 -0.095]
 [ 0.07   0.15   0.075 -0.295]
 [-0.13  -0.05  -0.125  0.305]
 [ 0.07  -0.45   0.075  0.305]
 [ 0.27  -0.05   0.075 -0.295]
 [-0.53   0.15   0.075  0.305]
 [ 0.07   0.15   0.075 -0.295]
 [ 0.27  -0.05  -0.125 -0.095]
 [-0.33   0.15   0.075  0.105]
 [ 0.07   0.15  -0.125 -0.095]
 [ 0.07  -0.05   0.075 -0.095]
 [ 0.27  -0.25  -0.125  0.105]
 [ 0.07  -0.05   0.075 -0.095]
 [-0.33   0.15  -0.125  0.305]
 [ 0.07   0.15   0.075 -0.295]
 [-0.13  -0.05   0.075  0.105]
 [ 0.27  -0.45   0.075  0.105]
 [ 0.07   0.15   0.075 -0.295]
 [-0.13   0.15  -0.125  0.105]
 [ 0.07  -0.05   0.075 -0.095]
 [ 0.07  -0.05  -0.125  0.105]
 [-0.13  -0.25   0.075  0.305]
 [ 0.07   0.15  -0.125 -0.095]
 [ 0.07  -0.05   0.075 -0.095]
 [-0.13   0.15  -0.125  0.105]
 [-0.13 

# Example Lottery Solution

In [10]:
df= pd.read_excel('DataForSim.xls')

In [13]:
df2=df.copy()

,Year,AdvNo,SlNo,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,alphaOBC,alphaSC,alphaST,alphaUR,DeptSeats
0,2005,1,1,3,1,1,0,0.27,0.15,0.075,0.505,5
1,2005,1,2,1,1,1,0,0.27,0.15,0.075,0.505,3
2,2005,1,3,3,1,1,0,0.27,0.15,0.075,0.505,5
3,2005,1,4,3,1,1,0,0.27,0.15,0.075,0.505,5
4,2005,1,5,3,2,2,0,0.27,0.15,0.075,0.505,7
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2013,9,20,1,0,0,0,0.27,0.15,0.075,0.505,1
212,2013,9,21,4,2,0,3,0.27,0.15,0.075,0.505,9
213,2013,9,22,9,4,0,4,0.27,0.15,0.075,0.505,17
214,2013,9,23,3,0,0,0,0.27,0.15,0.075,0.505,3


In [59]:
a=np.array([0.27,0.15,0.075,0.505])

list_SlNo=df2['SlNo'].unique()
for SlNo in list_SlNo:
    list_DeptSeats=list(df2[df2['SlNo']==SlNo]['DeptSeats'])
    roster=find_roster(a)
    cumulative_seats=0
    for i, DeptSeats in enumerate(list_DeptSeats):
        seat_allocation=roster[cumulative_seats:(cumulative_seats+DeptSeats)].sum(axis=0)
        cumulative_seats+=DeptSeats
        df_row_index=df2.index[(df2['Year']==(2005+i)) & (df2['SlNo']==SlNo)][0]
        
        df2.loc[df_row_index,'DeptSeatsOBC']=seat_allocation[0]
        df2.loc[df_row_index,'DeptSeatsSC']=seat_allocation[1]
        df2.loc[df_row_index,'DeptSeatsST']=seat_allocation[2]
        df2.loc[df_row_index,'DeptSeatsUR']=seat_allocation[3]
        
        

In [60]:
df2

,Year,AdvNo,SlNo,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,alphaOBC,alphaSC,alphaST,alphaUR,DeptSeats
0,2005,1,1,2.0,1.0,1.0,1.0,0.27,0.15,0.075,0.505,5
1,2005,1,2,2.0,1.0,0.0,0.0,0.27,0.15,0.075,0.505,3
2,2005,1,3,2.0,1.0,1.0,1.0,0.27,0.15,0.075,0.505,5
3,2005,1,4,2.0,2.0,1.0,0.0,0.27,0.15,0.075,0.505,5
4,2005,1,5,4.0,1.0,1.0,1.0,0.27,0.15,0.075,0.505,7
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2013,9,20,0.0,1.0,0.0,0.0,0.27,0.15,0.075,0.505,1
212,2013,9,21,5.0,3.0,1.0,0.0,0.27,0.15,0.075,0.505,9
213,2013,9,22,9.0,5.0,3.0,0.0,0.27,0.15,0.075,0.505,17
214,2013,9,23,2.0,1.0,0.0,0.0,0.27,0.15,0.075,0.505,3


df2.to_excel('Simulation.xls', index=False)

In [80]:
df2.sum()

Year            433944.000000
AdvNo             1080.000000
SlNo              2700.000000
DeptSeatsUR        588.000000
DeptSeatsOBC       312.000000
DeptSeatsSC        175.000000
DeptSeatsST         85.000000
alphaOBC            58.320002
alphaSC             32.400001
alphaST             16.200001
alphaUR            109.079999
DeptSeats         1160.000000
dtype: float64

In [81]:
a*1160

array([313.2, 174. ,  87. , 585.8])

# SIMULATION RESULT

In [2]:
df= pd.read_excel('DataForSim.xls')

In [3]:
a=np.array([0.27,0.15,0.075,0.505])

In [20]:
tol=0.0001
P_n=len(a)
P_m=1
while ( ( ((np.mod(P_m*a, 1) > tol) & (np.mod(P_m*a, 1) < (1-tol)) )  *1).sum() > 0):
    P_m+=1

T=50
num_dept=df['SlNo'].nunique()
roster=np.zeros(((T*num_dept),P_m,P_n))
for i in range(T*num_dept):
    roster[i]= find_roster(a)

In [24]:
# np.save('rosters_simulation', roster)

In [4]:
roster=np.load('rosters_simulation.npy')

In [26]:
list_SlNo=df['SlNo'].unique()
for t in range(T):
    df2=df.copy()
    for SlNo in list_SlNo:
        list_DeptSeats=list(df2[df2['SlNo']==SlNo]['DeptSeats'])
        dep_roster=roster[t*num_dept+SlNo-1]
        cumulative_seats=0
        for i, DeptSeats in enumerate(list_DeptSeats):
            seat_allocation=dep_roster[cumulative_seats:(cumulative_seats+DeptSeats)].sum(axis=0)
            cumulative_seats+=DeptSeats
            df_row_index=df2.index[(df2['Year']==(2005+i)) & (df2['SlNo']==SlNo)][0]

            df2.loc[df_row_index,'DeptSeatsOBC']=seat_allocation[0]
            df2.loc[df_row_index,'DeptSeatsSC']=seat_allocation[1]
            df2.loc[df_row_index,'DeptSeatsST']=seat_allocation[2]
            df2.loc[df_row_index,'DeptSeatsUR']=seat_allocation[3]

    df2.to_excel('Simulation{}.xls'.format((t+1)), index=False)    

<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df2.to_excel('Simulation{}.xls'.format((t+1)), index=False)
<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the wa

<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df2.to_excel('Simulation{}.xls'.format((t+1)), index=False)
<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the wa

<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df2.to_excel('Simulation{}.xls'.format((t+1)), index=False)
<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the wa

<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df2.to_excel('Simulation{}.xls'.format((t+1)), index=False)
<ipython-input-26-6771baf9e3f4>:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the wa

In [41]:
simulation_df=pd.read_excel('Simulation{}.xls'.format((t+1)))

In [42]:
simulation_df

,Year,AdvNo,SlNo,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,alphaOBC,alphaSC,alphaST,alphaUR,DeptSeats
0,2005,1,1,3,1,1,0,0.27,0.15,0.075,0.505,5
1,2005,1,2,1,1,1,0,0.27,0.15,0.075,0.505,3
2,2005,1,3,2,2,1,0,0.27,0.15,0.075,0.505,5
3,2005,1,4,3,1,1,0,0.27,0.15,0.075,0.505,5
4,2005,1,5,4,2,1,0,0.27,0.15,0.075,0.505,7
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2013,9,20,0,0,1,0,0.27,0.15,0.075,0.505,1
212,2013,9,21,4,3,1,1,0.27,0.15,0.075,0.505,9
213,2013,9,22,8,5,3,1,0.27,0.15,0.075,0.505,17
214,2013,9,23,2,1,0,0,0.27,0.15,0.075,0.505,3


In [46]:
for t in range(T):
    simulation_df=pd.read_excel('Simulation{}.xls'.format((t+1)))
    university_allocation=simulation_df.groupby('Year').sum()[['DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST', 'DeptSeats']]
    university_allocation_cum=simulation_df.groupby('Year').sum()[['DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST', 'DeptSeats']].cumsum()
    university_allocation.to_excel('Uni_Simu{}.xls'.format((t+1)), index=True)
    university_allocation_cum.to_excel('Cum_Uni_Simu{}.xls'.format((t+1)), index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Simulation1.xls'

In [18]:
df

,Year,AdvNo,SlNo,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,alphaOBC,alphaSC,alphaST,alphaUR,DeptSeats
0,2005,1,1,3,1,1,0,0.27,0.15,0.075,0.505,5
1,2005,1,2,1,1,1,0,0.27,0.15,0.075,0.505,3
2,2005,1,3,3,1,1,0,0.27,0.15,0.075,0.505,5
3,2005,1,4,3,1,1,0,0.27,0.15,0.075,0.505,5
4,2005,1,5,3,2,2,0,0.27,0.15,0.075,0.505,7
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2013,9,20,1,0,0,0,0.27,0.15,0.075,0.505,1
212,2013,9,21,4,2,0,3,0.27,0.15,0.075,0.505,9
213,2013,9,22,9,4,0,4,0.27,0.15,0.075,0.505,17
214,2013,9,23,3,0,0,0,0.27,0.15,0.075,0.505,3


# RESULT FOR THE PAPER

In [6]:
df= pd.read_excel('DataForSim.xls')

In [7]:
a=np.array([0.27,0.15,0.075,0.505])

In [20]:
tol=0.0001
P_n=len(a)
P_m=1
while ( ( ((np.mod(P_m*a, 1) > tol) & (np.mod(P_m*a, 1) < (1-tol)) )  *1).sum() > 0):
    P_m+=1

T=50
num_dept=df['SlNo'].nunique()
roster=np.zeros(((T*num_dept),P_m,P_n))
for i in range(T*num_dept):
    roster[i]= find_roster(a)

In [24]:
# np.save('rosters_simulation', roster)

In [8]:
roster=np.load('rosters_simulation.npy')

In [22]:
df_simulation=pd.DataFrame(columns=['SimulationNo', 'Year','DeptSeats',
                                    'DeptSeatsUR','DeptSeatsOBC',
                                    'DeptSeatsSC', 'DeptSeatsST','FairShareUR',
                                    'FairShareOBC','FairShareSC','FairShareST',
                                    'BiasUR','BiasOBC','BiasSC','BiasST'])
list_SlNo=df['SlNo'].unique()
num_dept=df['SlNo'].nunique()
T=50
for t in range(T):
    df2=df.copy()
    for SlNo in list_SlNo:
        list_DeptSeats=list(df2[df2['SlNo']==SlNo]['DeptSeats'])
        dep_roster=roster[t*num_dept+SlNo-1]
        cumulative_seats=0
        for i, DeptSeats in enumerate(list_DeptSeats):
            seat_allocation=dep_roster[cumulative_seats:(cumulative_seats+DeptSeats)].sum(axis=0)
            cumulative_seats+=DeptSeats
            df_row_index=df2.index[(df2['Year']==(2005+i)) & (df2['SlNo']==SlNo)][0]

            df2.loc[df_row_index,'DeptSeatsOBC']=seat_allocation[0]
            df2.loc[df_row_index,'DeptSeatsSC']=seat_allocation[1]
            df2.loc[df_row_index,'DeptSeatsST']=seat_allocation[2]
            df2.loc[df_row_index,'DeptSeatsUR']=seat_allocation[3]

    df2=df2.groupby('Year').sum()[['DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST', 'DeptSeats']].cumsum().reset_index()
    
    df2['SimulationNo']=t+1
    
    df2=df2[['SimulationNo', 'Year','DeptSeats','DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST' ]]
    
    df2['FairShareUR']=df2['DeptSeats']*0.505
    df2['FairShareOBC']=df2['DeptSeats']*0.27
    df2['FairShareSC']=df2['DeptSeats']*0.15
    df2['FairShareST']=df2['DeptSeats']*0.075
    
    df2['BiasUR']= df2['DeptSeatsUR'] - df2['FairShareUR']
    df2['BiasOBC']= df2['DeptSeatsOBC'] - df2['FairShareOBC']
    df2['BiasSC']= df2['DeptSeatsSC'] - df2['FairShareSC']
    df2['BiasST']= df2['DeptSeatsST'] - df2['FairShareST']
    df_simulation=df_simulation.append(df2)

In [35]:
df_simulation.to_excel('Simulations.xls')

<ipython-input-35-fc7440fc2d98>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_simulation.to_excel('Simulations.xls')


In [25]:
df_simulation.groupby('Year').mean()

,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,FairShareUR,FairShareOBC,FairShareSC,FairShareST,BiasUR,BiasOBC,BiasSC,BiasST
Year,,,,,,,,,,,,
2005,44.22,23.72,12.84,6.22,43.935,23.49,13.05,6.525,0.285,0.23,-0.21,-0.305
2006,88.04,46.82,26.04,13.10,87.870,46.98,26.10,13.050,0.170,-0.16,-0.06,0.050
2007,131.74,70.78,38.98,19.50,131.805,70.47,39.15,19.575,-0.065,0.31,-0.17,-0.075
2008,175.48,94.02,52.28,26.22,175.740,93.96,52.20,26.100,-0.260,0.06,0.08,0.120
2009,242.02,129.16,71.88,35.94,241.895,129.33,71.85,35.925,0.125,-0.17,0.03,0.015
2010,317.76,170.04,94.50,47.70,318.150,170.10,94.50,47.250,-0.390,-0.06,0.00,0.450
2011,403.64,216.00,120.02,60.34,404.000,216.00,120.00,60.000,-0.360,0.00,0.02,0.340
2012,494.44,264.78,146.92,73.86,494.900,264.60,147.00,73.500,-0.460,0.18,-0.08,0.360
2013,585.34,313.28,173.70,87.68,585.800,313.20,174.00,87.000,-0.460,0.08,-0.30,0.680


In [28]:
df_simulation.groupby('Year').max()

,SimulationNo,DeptSeats,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,FairShareUR,FairShareOBC,FairShareSC,FairShareST,BiasUR,BiasOBC,BiasSC,BiasST
Year,,,,,,,,,,,,,,
2005,50,87,47.0,27.0,18.0,12.0,43.935,23.49,13.05,6.525,3.065,3.51,4.95,5.475
2006,50,174,90.0,50.0,32.0,16.0,87.870,46.98,26.10,13.050,2.130,3.02,5.90,2.950
2007,50,261,134.0,74.0,43.0,25.0,131.805,70.47,39.15,19.575,2.195,3.53,3.85,5.425
2008,50,348,177.0,99.0,58.0,31.0,175.740,93.96,52.20,26.100,1.260,5.04,5.80,4.900
2009,50,479,245.0,133.0,76.0,39.0,241.895,129.33,71.85,35.925,3.105,3.67,4.15,3.075
2010,50,630,322.0,175.0,99.0,52.0,318.150,170.10,94.50,47.250,3.850,4.90,4.50,4.750
2011,50,800,407.0,221.0,124.0,65.0,404.000,216.00,120.00,60.000,3.000,5.00,4.00,5.000
2012,50,980,499.0,270.0,151.0,78.0,494.900,264.60,147.00,73.500,4.100,5.40,4.00,4.500
2013,50,1160,590.0,319.0,178.0,91.0,585.800,313.20,174.00,87.000,4.200,5.80,4.00,4.000


In [29]:
df_simulation.groupby('Year').min()

,SimulationNo,DeptSeats,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,FairShareUR,FairShareOBC,FairShareSC,FairShareST,BiasUR,BiasOBC,BiasSC,BiasST
Year,,,,,,,,,,,,,,
2005,1,87,41.0,18.0,7.0,1.0,43.935,23.49,13.05,6.525,-2.935,-5.49,-6.05,-5.525
2006,1,174,87.0,42.0,22.0,9.0,87.870,46.98,26.10,13.050,-0.870,-4.98,-4.10,-4.050
2007,1,261,127.0,67.0,34.0,16.0,131.805,70.47,39.15,19.575,-4.805,-3.47,-5.15,-3.575
2008,1,348,174.0,88.0,50.0,22.0,175.740,93.96,52.20,26.100,-1.740,-5.96,-2.20,-4.100
2009,1,479,237.0,125.0,68.0,33.0,241.895,129.33,71.85,35.925,-4.895,-4.33,-3.85,-2.925
2010,1,630,313.0,166.0,90.0,43.0,318.150,170.10,94.50,47.250,-5.150,-4.10,-4.50,-4.250
2011,1,800,398.0,212.0,117.0,55.0,404.000,216.00,120.00,60.000,-6.000,-4.00,-3.00,-5.000
2012,1,980,490.0,261.0,143.0,71.0,494.900,264.60,147.00,73.500,-4.900,-3.60,-4.00,-2.500
2013,1,1160,580.0,308.0,170.0,82.0,585.800,313.20,174.00,87.000,-5.800,-5.20,-4.00,-5.000


In [32]:
df_simulation.groupby('Year').quantile([0,0.25,0.5, 0.75,1])

DeptSeatsUR  DeptSeatsOBC  DeptSeatsSC  DeptSeatsST  FairShareUR  \
Year                                                                          
2005 0.00         41.0         18.00         7.00         1.00       43.935   
     0.25         43.0         23.00        11.00         5.00       43.935   
     0.50         44.0         24.00        13.00         6.00       43.935   
     0.75         46.0         25.00        15.00         7.00       43.935   
     1.00         47.0         27.00        18.00        12.00       43.935   
2006 0.00         87.0         42.00        22.00         9.00       87.870   
     0.25         87.0         46.00        25.00        12.00       87.870   
     0.50         88.0         47.00        26.00        13.00       87.870   
     0.75         89.0         48.00        27.75        14.00       87.870   
     1.00         90.0         50.00        32.00        16.00       87.870   
2007 0.00        127.0         67.00        34.00        16.00      131.805   
     0.25        131.0         70.00        38.00        18.00      131.805   
     0.50        132.0         71.00        39.00        19.00      131.805   
     0.75        133.0         72.00        40.00        21.00      131.805   
     1.00        134.0         74.00        43.00        25.00      131.805   
2008 0.00        174.0         88.00        50.00        22.00      175.740   
     0.25        175.0         92.25        51.00        25.00      175.740   
     0.50        175.0         94.00        52.00        26.00      175.740   
     0.75        176.0         95.75        53.00        27.75      175.740   
     1.00        177.0         99.00        58.00        31.00      175.740   
2009 0.00        237.0        125.00        68.00        33.00      241.895   
     0.25        241.0        128.00        70.25        35.00      241.895   
     0.50        242.0        129.00        71.50        36.00      241.895   
     0.75        243.0        131.00        73.00        37.00      241.895   
     1.00        245.0        133.00        76.00        39.00      241.895   
2010 0.00        313.0        166.00        90.00        43.00      318.150   
     0.25        317.0        169.00        94.00        47.00      318.150   
     0.50        318.0        170.00        94.50        48.00      318.150   
     0.75        319.0        171.00        96.00        49.00      318.150   
     1.00        322.0        175.00        99.00        52.00      318.150   
2011 0.00        398.0        212.00       117.00        55.00      404.000   
     0.25        403.0        215.00       119.00        59.00      404.000   
     0.50        404.0        216.00       120.00        60.00      404.000   
     0.75        405.0        217.75       121.00        61.00      404.000   
     1.00        407.0        221.00       124.00        65.00      404.000   
2012 0.00        490.0        261.00       143.00        71.00      494.900   
     0.25        493.0        263.00       146.00        72.00      494.900   
     0.50        494.0        265.00       147.00        74.00      494.900   
     0.75        495.0        266.00       148.75        75.00      494.900   
     1.00        499.0        270.00       151.00        78.00      494.900   
2013 0.00        580.0        308.00       170.00        82.00      585.800   
     0.25        584.0        312.00       172.00        86.25      585.800   
     0.50        585.0        313.00       174.00        88.00      585.800   
     0.75        587.0        314.00       175.00        89.00      585.800   
     1.00        590.0        319.00       178.00        91.00      585.800   

           FairShareOBC  FairShareSC  FairShareST  BiasUR  BiasOBC  BiasSC  \
Year                                                                         
2005 0.00         23.49        13.05        6.525  -2.935    -5.49   -6.05   
     0.25         23.49        13.05        6.525  -0.935    -0

In [27]:
a*87

array([23.49 , 13.05 ,  6.525, 43.935])

In [9]:
df2=df.copy()

In [17]:
df2=df2[['SimulationNo', 'Year','DeptSeats','DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST' ]]
df2

,SimulationNo,Year,DeptSeats,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST
0,2,2005,87,45,22,20,0
1,2,2006,174,90,44,40,0
2,2,2007,261,135,66,60,0
3,2,2008,348,180,88,80,0
4,2,2009,479,246,126,98,9
5,2,2010,630,322,167,121,20
6,2,2011,800,407,213,147,33
7,2,2012,980,501,258,175,46
8,2,2013,1160,591,307,202,60


In [33]:
df2.loc[0:23,['DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST', 'DeptSeats']].sum()

DeptSeatsUR     45.0
DeptSeatsOBC    22.0
DeptSeatsSC     13.0
DeptSeatsST      7.0
DeptSeats       87.0
dtype: float64

In [12]:
df2=df2.groupby('Year').sum()[['DeptSeatsUR','DeptSeatsOBC','DeptSeatsSC', 'DeptSeatsST', 'DeptSeats']].cumsum().reset_index()

In [ ]:
s.cumsum()


In [16]:
df2

,Year,DeptSeatsUR,DeptSeatsOBC,DeptSeatsSC,DeptSeatsST,DeptSeats,SimulationNo
0,2005,45,22,20,0,87,2
1,2006,90,44,40,0,174,2
2,2007,135,66,60,0,261,2
3,2008,180,88,80,0,348,2
4,2009,246,126,98,9,479,2
5,2010,322,167,121,20,630,2
6,2011,407,213,147,33,800,2
7,2012,501,258,175,46,980,2
8,2013,591,307,202,60,1160,2


df2.to_excel('Simulation.xls', index=False)

In [80]:
df2.sum()

Year            433944.000000
AdvNo             1080.000000
SlNo              2700.000000
DeptSeatsUR        588.000000
DeptSeatsOBC       312.000000
DeptSeatsSC        175.000000
DeptSeatsST         85.000000
alphaOBC            58.320002
alphaSC             32.400001
alphaST             16.200001
alphaUR            109.079999
DeptSeats         1160.000000
dtype: float64

In [81]:
a*1160

array([313.2, 174. ,  87. , 585.8])